# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
from azureml.core import Workspace, Experiment
from azureml.core import Dataset
import pandas as pd

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()

experiment_name = 'heart_tree_hyper'

experiment=Experiment(ws, experiment_name)

In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute


compute_target = ComputeTarget(workspace=ws, name="ds1v2-cluster")
compute_target.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [34]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, normal, choice
from azureml.core import ScriptRunConfig
from azureml.core import Environment

import os

early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)


param_sampling = RandomParameterSampling( {
        "criterion": choice("gini","entropy"),
        "splitter": choice("best", "random"),
        "max_depth": choice(250,110,10,15,60,18,20,22,24,28,30,40,50,35,55,45,70,80,90,100,150,200,85,300)
    })

sklearn_env = Environment.from_conda_specification(name="project_environment",file_path="conda_env.yml")
if "outputs" not in os.listdir():
    os.mkdir("./outputs")
if "training" not in os.listdir():
    os.mkdir("./training")

estimator = ScriptRunConfig(source_directory=".",script="decision_tree.py",compute_target=compute_target,environment=sklearn_env)

hyperdrive_run_config = HyperDriveConfig(run_config=estimator,
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=100,
                             max_concurrent_runs=4)




In [38]:
hyperdrive_run = experiment.submit(hyperdrive_run_config)


In [7]:
from azureml.train.hyperdrive import HyperDriveRun
hyperdrive_run = HyperDriveRun(experiment, run_id= "HD_b5a741aa-60ad-4475-ac81-8ac9b90f6bf5", hyperdrive_config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [39]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [8]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n parameter values: ', parameter_values)
### YOUR CODE HERE ###

print(best_run.get_file_names())



Best Run Id:  HD_b5a741aa-60ad-4475-ac81-8ac9b90f6bf5_99

 Accuracy: 0.9272727272727272

 parameter values:  ['--criterion', 'gini', '--max_depth', '40', '--splitter', 'random']
['azureml-logs/55_azureml-execution-tvmps_cc77e850b2db75c4262aa0ad0d54226b3d27656ae7e8a5b61395d5769a9daf5b_p.txt', 'azureml-logs/65_job_prep-tvmps_cc77e850b2db75c4262aa0ad0d54226b3d27656ae7e8a5b61395d5769a9daf5b_p.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_cc77e850b2db75c4262aa0ad0d54226b3d27656ae7e8a5b61395d5769a9daf5b_p.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/90_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/hyperdrive-model.joblib']


In [9]:
#TODO: Save the best model
best_run.download_file(best_run.get_file_names()[-1], output_file_path='./outputs/')

In [12]:
print(best_run.get_file_names())

['azureml-logs/55_azureml-execution-tvmps_cc77e850b2db75c4262aa0ad0d54226b3d27656ae7e8a5b61395d5769a9daf5b_p.txt', 'azureml-logs/65_job_prep-tvmps_cc77e850b2db75c4262aa0ad0d54226b3d27656ae7e8a5b61395d5769a9daf5b_p.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_cc77e850b2db75c4262aa0ad0d54226b3d27656ae7e8a5b61395d5769a9daf5b_p.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/90_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/hyperdrive-model.joblib']


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [10]:
best_run.register_model(model_name="best_hd_model", model_path='hyperdrive-model.joblib')

Model(workspace=Workspace.create(name='SkynetML', subscription_id='17e15d04-6670-44fd-bf24-195e65650e31', resource_group='ML'), name=best_hd_model, id=best_hd_model:1, version=1, tags={}, properties={})

In [11]:
from azureml.core.model import Model
model = Model(ws,"best_hd_model")
print(model)

Model(workspace=Workspace.create(name='SkynetML', subscription_id='17e15d04-6670-44fd-bf24-195e65650e31', resource_group='ML'), name=best_hd_model, id=best_hd_model:1, version=1, tags={}, properties={})


In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
service_name = "hyperdrive-model-service"
inference_config = InferenceConfig(entry_script="./score.py", environment=sklearn_env)
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 2)
service = Model.deploy(workspace=ws,
                          name=service_name,
                          models=[model],
                          inference_config=inference_config,
                          deployment_config=deployment_config,
                          overwrite=True)
                          
service.wait_for_deployment(show_output=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [29]:
import requests
import json


scoring_uri = 'http://b5f0f797-f0d3-4647-af72-4462aa93ff41.eastus.azurecontainer.io/score'

data = {
  "Inputs":{
      "data":
        [
          {
            "age": 50,
            "anaemia": 1,
            "creatinine": 452,
            "diabetes": 0,
            "ejection_fraction": 35,
            "high_blood_pressure": 1,
            "platelets": 196000,
            "serum_creatinine": 1.1,
            "serum_sodium": 130,
            "sex": 0,
            "smoking": 1,
            "time": 150
          },
          {
            "age": 45,
            "anaemia": 0,
            "creatinine": 352,
            "diabetes": 1,
            "ejection_fraction": 60,
            "high_blood_pressure": 0,
            "platelets": 200000,
            "serum_creatinine": 1.9,
            "serum_sodium": 123,
            "sex": 1,
            "smoking": 0,
            "time": 100
          }
      ]
  },
  "method":"predict"

    }
    

input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)


headers = {'Content-Type': 'application/json'}

resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

{"Results": [0, 1]}


TODO: In the cell below, print the logs of the web service and delete the service

In [30]:
from azureml.core import Workspace
from azureml.core.webservice import Webservice



ws = Workspace.from_config()


name = "hyperdrive-model-service"

# load existing web service
service = Webservice(name=name, workspace=ws)
service.update(enable_app_insights=True)
logs = service.get_logs()

for line in logs.split('\n'):
    print(line)

2021-09-22T23:01:51,116739000+00:00 - rsyslog/run 
2021-09-22T23:01:51,118837400+00:00 - iot-server/run 
2021-09-22T23:01:51,131639000+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-09-22T23:01:51,191222300+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_732632de80c129004c31329a710852e7/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-09-22T23:01:51,717596200+00:00 - iot-server/finish 1 0
2021-09-22T23:01:51,720308500+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (67)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 95
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2021-09-22 23:01:54,820 | root | INFO | Starting up app insights client
logging socket was 

In [40]:
import onnxmltools
import joblib
from skl2onnx.common.data_types import FloatTensorType, Int64TensorType
# loading best model from outputs folder
model = joblib.load("./outputs/hyperdrive-model.joblib")
#defining intial type parameter. The model takes 12 integers and 1 float as input, workaround it to
# declare all as float since it will not matter to the model
initial_type =[('float_input', FloatTensorType([None,13]))]
#using onnxmltools function convert_sklearn to convert the decision tree to onnx model
onxmdl = onnxmltools.convert_sklearn(model, name="onxmdl", initial_types=initial_type)
print(onxmdl)

Trying to unpickle estimator DecisionTreeClassifier from version 0.24.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
The maximum opset needed by this model is only 1.
The maximum opset needed by this model is only 9.


ir_version: 7
producer_name: "skl2onnx"
producer_version: "1.4.9"
domain: "ai.onnx"
model_version: 0
doc_string: ""
graph {
  node {
    input: "float_input"
    output: "label"
    output: "probabilities"
    name: "TreeEnsembleClassifier"
    op_type: "TreeEnsembleClassifier"
    attribute {
      name: "class_ids"
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      int